In [1]:
# Mounting Google Drive
from google.colab import drive
from os.path import join
ROOT = '/content/drive'
print(ROOT)
drive.mount(ROOT)

/content/drive
Mounted at /content/drive


In [2]:
path = '/content/drive/My Drive/Colab Notebooks/ProjectData/FaceRecognition'
vgg16_path =['/content/drive/My Drive/Colab Notebooks/ProjectData/FaceRecognition/pickle/transferVGG16_2_celeb_dataset.h5',
             '/content/drive/My Drive/Colab Notebooks/ProjectData/FaceRecognition/pickle/checkpoint/face_VGG16_2.hdf5'] 
inceptionV3_path = ['/content/drive/My Drive/Colab Notebooks/ProjectData/FaceRecognition/pickle/transferInceptionV3_celeb_dataset.h5',
                    '/content/drive/My Drive/Colab Notebooks/ProjectData/FaceRecognition/pickle/checkpoint/face_inceptionv3_3.hdf5']
myCnn_path = ['/content/drive/My Drive/Colab Notebooks/ProjectData/FaceRecognition/pickle/aug/myOwnCNN_celeb_dataset.h5',
              '/content/drive/My Drive/Colab Notebooks/ProjectData/FaceRecognition/pickle/aug/myOwnCNN_celeb_dataset_checkpoint_weights.hdf5']
caffe_path = ['/content/drive/My Drive/Colab Notebooks/ProjectData/FaceRecognition/pickle/CV2_Caffemodel/celeb/recognizer_dataset_celeb.pickle']
testVid = '/content/drive/My Drive/Colab Notebooks/ProjectData/FaceRecognition/testset/Bolly.avi'
outputVid = '/content/drive/My Drive/Colab Notebooks/ProjectData/FaceRecognition/testset/BollyOutputVid.avi'
face_detection_path= "/content/drive/My Drive/Colab Notebooks/ProjectData/FaceRecognition/face_detection_model/res10_300x300_ssd_iter_140000.caffemodel"
proto_path = "/content/drive/My Drive/Colab Notebooks/ProjectData/FaceRecognition/face_detection_model/deploy.prototxt"

In [3]:
import keras
import pickle
import cv2

In [4]:
detector = cv2.dnn.readNetFromCaffe(proto_path, face_detection_path)

In [5]:
caffe_model = pickle.load(open(caffe_path[0], 'rb'))

In [6]:
inception_model =  keras.models.load_model(inceptionV3_path[0])
inception_model.load_weights(inceptionV3_path[1])

In [7]:
vgg16_model = keras.models.load_model(vgg16_path[0])
vgg16_model.load_weights(vgg16_path[1])

In [8]:
myCnn_model = keras.models.load_model(myCnn_path[0])
myCnn_model.load_weights(myCnn_path[1])

In [9]:

def getFace_CV2DNN(image):
  (h,w) = image.shape[:2]
  blob = cv2.dnn.blobFromImage(cv2.resize(image, (300,300)),1.0, (300,300),(104.0, 177.0, 123.0), swapRB= False, crop = False)
  detector.setInput(blob)
  detections = detector.forward()
  max = -1
  fH = 0
  fW = 0
  x1 = 0
  x2=0
  y1=0
  y2 = 0
  for i in range(0,detections.shape[2]):
    confidence = detections[0,0,i,2];

    if confidence < 0.7:
      continue

    box = detections[0, 0, i, 3:7] * np.array([w, h, w, h])
    (startX, startY, endX, endY) = box.astype("int")
    cv2.rectangle(image, (startX, startY), (endX, endY), (0,0,255), 2)
    roi = image[startY:endY, startX:endX]
    setlabels(roi,image, startX, startY)
    fH = endX - startX
    fW = endY - startY
    #print("ggg-",fW,fH)
    if fH < 20 or fW < 20:
      continue
    area = fH*fW
    if max < area:
      max = area
      x1 = startX
      x2 = endX
      y1 = startY
      y2 = endY
  fHM = x2-x1
  fWM = y2 - y1
  return [(x1,y1,fHM,fWM)]


In [10]:
vgg16_model.summary()

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0   

In [13]:
def setlabels(face,image, x, y):
  
  
  #cv2_imshow(face)
  #print(face.shape)
  if(face.shape == (0,0,3)):
    return
  im = cv2.resize(face, (224, 224)).astype(np.float32)
  im = im.transpose((2,0,1))
  #im = np.expand_dims(im, axis=3)
  im = im.reshape(1,224,224,3)
  #print(im.shape)
  out = inception_model.predict(im)
  label = np.argmax(out)
  label = 'Name'+str(label)
  #print(label)


  cv2.putText(image, label, (x,y), cv2.FONT_HERSHEY_COMPLEX, 1, (255,0,0), 2, cv2.LINE_AA)
  

In [14]:
import cv2
from google.colab.patches import cv2_imshow
import matplotlib.pyplot as plt
import numpy as np

cap = cv2.VideoCapture(testVid)


#A video writer
fourcc = cv2.VideoWriter_fourcc(*'XVID')
# output file stream for video
out  = cv2.VideoWriter(outputVid,fourcc, 20.0, (480,640))

while cap.isOpened():
    ret, frame = cap.read()
    if (not ret):
        break
    cords = getFace_CV2DNN(frame)
    #setlabels(frame, cords)
    #cv2_imshow(frame)
    video = cv2.resize(frame,(480,640))
    out.write(video)


    ''' 
    #cv2_imshow(frame)
    if ret == False:
      print('ll')
    gray =  cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    cv2_imshow( gray)
    
    # writing to video out stream
    gray = np.stack((gray,)*3,axis=-1)
    out.write(gray)
    '''
cap.release()
out.release()

